# Rules to select the most representative error message from WMArchive entries

<b> Problem </b>: The WMArchive entries can have multiple steps with multiple error messages. Due to memory constraints of the GPU the <b> most representative message </b> has to be picked for the machine learning.

In [1]:
import pandas as pd
import numpy as np

### Load the filtered WMArchive messages

In [3]:
tokens = pd.read_hdf('test.h5', 'frame')

This pandas frame contains the error messages written out from WMArchive. Each row corresponds to a WMArchive entry and each of the keys ('task_name', 'error', 'site') is unique and can be associated with the counts and labels from the console. The field 'error' is the error that is reported by the console. The fields 'exit_codes', 'error_msg', 'error_type', 'steps_counter' and 'names' correspond to the message sequence in the WMArchive entry. The field 'error' that is returned by the console appears at least one time in the 'exit_codes' of the WMArchive message sequence. For each of the sequences only one message should represent the error.

In [4]:
tokens.head()

,task_name,error,site,exit_codes,error_msg,error_type,steps_counter,names
0,/amaltaro_Run2018A-v1-DoubleMuon-17Sep2018_102...,85,T1_UK_RAL,"[99996, 85, 8021, 99999, 85]","[Failed to find a step report for stageOut1!, ...","[ReportManipulatingError, CMSSWStepFailure, Fa...","[0, 2, 2, 2, 2]","[stageOut1, cmsRun1, cmsRun1, cmsRun1, cmsRun1]"
1,/amaltaro_Run2018A-v1-DoubleMuon-17Sep2018_102...,50664,T2_DE_RWTH,"[50664, 99996, 143, 50115, 99999, 143]",[Error in CMSSW step cmsRun1 Number of Cores: ...,"[PerformanceKill, ReportManipulatingError, CMS...","[0, 1, 3, 3, 3, 3]","[PerformanceError, stageOut1, cmsRun1, cmsRun1..."
2,/amaltaro_Run2018A-v1-DoubleMuon-17Sep2018_102...,50664,T2_DE_RWTH,"[50664, 99996, 143, 50115, 99999, 143]",[Error in CMSSW step cmsRun1 Number of Cores: ...,"[PerformanceKill, ReportManipulatingError, CMS...","[0, 1, 3, 3, 3, 3]","[PerformanceError, stageOut1, cmsRun1, cmsRun1..."
3,/amaltaro_Run2018A-v1-DoubleMuon-17Sep2018_102...,50664,T2_DE_RWTH,"[50664, 99996, 143, 50115, 99999, 143]",[Error in CMSSW step cmsRun1 Number of Cores: ...,"[PerformanceKill, ReportManipulatingError, CMS...","[0, 1, 3, 3, 3, 3]","[PerformanceError, stageOut1, cmsRun1, cmsRun1..."
4,/amaltaro_Run2018A-v1-DoubleMuon-17Sep2018_102...,99400,NoReportedSite,[99400],[Could not find jobReport ========== condor.2...,[RemovedByGLIDEIN],[0],[RemovedByGLIDEIN]


### Sequences per WMArchive entry

To get an overview over the types of sequences that appear, the occurenes are counted and some samples are checked

In [40]:
# Count the ocurrences of the sequences
tokens['error_type_string'] = [','.join(map(str, l)) for l in tokens['error_type']]
sequences = tokens['error_type_string'].value_counts().rename_axis('sequences').reset_index(name='counts')

In [41]:
pd.options.display.max_colwidth = 1000
sequences.head(50)

,sequences,counts
0,"ReportManipulatingError,CMSSWStepFailure,Fatal Exception,ErrorLoggingAddition,WMAgentStepExecutionError",32804
1,NoJobReport,12543
2,PerformanceKill,11802
3,"ReportManipulatingError,CMSSWStepFailure,Fatal Exception,ErrorLoggingAddition",10443
4,"ReportManipulatingError,Fatal Exception,Unknown,WMAgentStepExecutionError",9554
5,"ReportManipulatingError,CMSSWStepFailure,BadFWJRXML,ErrorLoggingAddition,WMAgentStepExecutionError",5839
6,"PerformanceKill,ReportManipulatingError,CMSSWStepFailure,BadFWJRXML,ErrorLoggingAddition,WMAgentStepExecutionError",5306
7,"ReportManipulatingError,Fatal Exception,CMSException,WMAgentStepExecutionError",3394
8,"PerformanceKill,ReportManipulatingError,BadFWJRXML,CMSSWStepFailure,ErrorLoggingAddition,WMAgentStepExecutionError",2880
9,"PerformanceKill,ReportManipulatingError,CMSSWStepFailure,BadFWJRXML,ErrorLoggingAddition",2338


In [8]:
# Function to print an exemplary sequence
def test_msg(tokens, seq, i_key):
    
    tokens_seq = tokens[tokens['error_type_string'] == seq]
    test = tokens_seq.iloc[i_key]
    
    print
    print 'Error code from console:', test['error'], 'Site from console:', test['site']
    print
    for i in range(len(test['exit_codes'])):
        
        exit_codes = test['exit_codes']
        steps = test['steps_counter']
        names = test['names']
        error_type = test['error_type']
        error_msg = test['error_msg']
        
        print
        print 'Exit code:', exit_codes[i], 'Step:', steps[i], 'Name:', names[i], 'Type:', error_type[i]
        print 
        print error_msg[i]
        print

### 1. Rule: Choose the first 'Fatal Exception'

Many message sequences contain 'Fatal Exception' in the cmsRun1 step. In most of the cases the error code that is given by the console corresponds to the 'CMSSWStepFailure' message. The CMSSWStepFailure contains the last 25 lines of the CMSSW stdout. After checking some samples, it seems that the important information is in the first 'Fatal Exception'. Frequently appearing snippets like 'ReportManipulatingError', 'ErrorLoggingAddition' and 'WMAgentStepExecutionError' seem to be very similar in most of the cases and don't contain much information about the error. While the last 25 lines in CMSSW stdout seem to contain quite often the Fatal Exception, I think it is cleaner to simply use only the first 'Fatal Exception' snippet as representation of the error. The 'CMSSWStepFailure' seems to report the bash code, while the Fatal Exception seems to report the CMMSW code. https://twiki.cern.ch/twiki/bin/view/CMSPublic/JobExitCodes

In [42]:
# Some examples
seq1 = 'ReportManipulatingError,CMSSWStepFailure,Fatal Exception,ErrorLoggingAddition,WMAgentStepExecutionError'
seq2 = 'ReportManipulatingError,CMSSWStepFailure,Fatal Exception,ErrorLoggingAddition'
seq3 = 'ReportManipulatingError,Fatal Exception,TypeError,WMAgentStepExecutionError'
seq4 = 'ReportManipulatingError,Fatal Exception,UnknownException'
seq5 = 'ReportManipulatingError,Fatal Exception,CMSException'
seq6 = 'ReportManipulatingError,Fatal Exception,UnknownException,WMAgentStepExecutionError'
seq7 = 'ReportManipulatingError,CMSSWStepFailure,Fatal Exception,Fatal Exception,ErrorLoggingAddition,WMAgentStepExecutionError'
seq8 = 'ReportManipulatingError,LogArchiveFailure,CMSSWStepFailure,Fatal Exception,ErrorLoggingAddition,WMAgentStepExecutionError'
seq9 = 'ReportManipulatingError,Fatal Exception,StdException,WMAgentStepExecutionError'
seq10 = 'PerformanceKill,ReportManipulatingError,CMSSWStepFailure,BadFWJRXML,ErrorLoggingAddition,WMAgentStepExecutionError,CMSSWStepFailure,Fatal Exception,WMAgentStepExecutionError'

test_msg(tokens, seq1, 100)


Error code from console: 92 Site from console: T1_US_FNAL_Disk


Exit code: 99996 Step: 0 Name: stageOut1 Type: ReportManipulatingError

Failed to find a step report for stageOut1!


Exit code: 92 Step: 2 Name: cmsRun1 Type: CMSSWStepFailure

  Adding last 25 lines of CMSSW stdout: %MSG ----- Begin Fatal Exception 04-Apr-2018 01:41:34 UTC----------------------- An exception of category 'FallbackFileOpenError' occurred while    [0] Constructing the EventProcessor    [1] Constructing input source of type PoolSource    [2] Calling RootFileSequenceBase::initTheFile()    [3] Calling StorageFactory::open()    [4] Calling XrdFile::open() Exception Message: Failed to open the file 'root://cmsxrootd.fnal.gov//store/data/Run2017C/BTagCSV/AOD/17Nov2017-v1/60001/644A998E-DBEB-E711-B703-02163E00C9C8.root'    Additional Info:       [a] XrdCl::File::Open(name='root://cmsxrootd-site.fnal.gov//store/data/Run2017C/BTagCSV/AOD/17Nov2017-v1/60001/644A998E-DBEB-E711-B703-02163E00C9C8.root', flags=0x10, pe

## 2. Rule: PerformanceKill 

If the error sequence contains 'PerformanceKill' the other snippets seem to not contain much additional information. Therefore only the short message of the 'Performance Kill' is chosen.

In [27]:
seq1 = 'PerformanceKill,ReportManipulatingError,CMSSWStepFailure,BadFWJRXML,ErrorLoggingAddition,WMAgentStepExecutionError'
seq2 = 'PerformanceKill,ReportManipulatingError,CMSSWStepFailure,BadFWJRXML,ErrorLoggingAddition'
seq3 = 'PerformanceKill,ReportManipulatingError,BadFWJRXML,CMSSWStepFailure,ErrorLoggingAddition,WMAgentStepExecutionError'
seq4 = 'PerformanceKill,ReportManipulatingError,CMSSWStepFailure,BadFWJRXML,ErrorLoggingAddition'
seq5 = 'PerformanceKill,ReportManipulatingError,CMSSWStepFailure,Fatal Exception,ErrorLoggingAddition,WMAgentStepExecutionError'
test_msg(tokens, seq4, 20)


Error code from console: 50664 Site from console: T1_US_FNAL_Disk


Exit code: 50664 Step: 0 Name: PerformanceError Type: PerformanceKill

Error in CMSSW step cmsRun1 Job has been running for more than: 159600 Job has been running for: 159639.017875 


Exit code: 99999 Step: 1 Name: stageOut1 Type: ReportManipulatingError

Could not find report file for step stageOut1!


Exit code: 143 Step: 3 Name: cmsRun1 Type: CMSSWStepFailure

  Adding last 25 lines of CMSSW stdout: #0  0x00000031a24df383 in poll () from /lib64/libc.so.6 #1  0x00002b338b7dc1b4 in full_read.constprop () from /cvmfs/cms.cern.ch/slc6_amd64_gcc530/cms/cmssw/CMSSW_8_0_29/lib/slc6_amd64_gcc530/pluginFWCoreServicesPlugins.so #2  0x00002b338b7dc41a in edm::service::InitRootHandlers::stacktraceFromThread() () from /cvmfs/cms.cern.ch/slc6_amd64_gcc530/cms/cmssw/CMSSW_8_0_29/lib/slc6_amd64_gcc530/pluginFWCoreServicesPlugins.so #3  0x00002b338b7dc504 in sig_dostack_then_abort () from /cvmfs/cms.cern.ch/slc6_amd64_gcc530/cms/c

## 3. Rule: both PerformanceKill and Fatal Exception ?

In some cases there seem to be both a performance kill and additionally a Fatal Exception. This is confusing.

In [28]:
seq1 = 'PerformanceKill,ReportManipulatingError,CMSSWStepFailure,Fatal Exception,ErrorLoggingAddition,WMAgentStepExecutionError'
test_msg(tokens, seq1, 20)


Error code from console: 50664 Site from console: T1_UK_RAL


Exit code: 50664 Step: 0 Name: PerformanceError Type: PerformanceKill

Error in CMSSW step cmsRun1 Number of Cores: 6 Job has been running for more than: 159600 Job has been running for: 159625.671447 


Exit code: 99996 Step: 1 Name: stageOut1 Type: ReportManipulatingError

Failed to find a step report for stageOut1!


Exit code: 134 Step: 3 Name: cmsRun1 Type: CMSSWStepFailure

  Adding last 25 lines of CMSSW stdout:  - Avg event:   9367.73  - Total loop:  9462.24  - Total init:  10151.5  - Total job:   19621.4  Event Throughput: 0.000634099 ev/s  CPU Summary:   - Total loop:  279.677  - Total init:  304.558  - Total extra: 0  - Total job:   591.458  [2019-01-31 15:31:33.540638 +0000][Error  ][XRootD            ] [lxfsre40c01.cern.ch:1095] Unable to get the response to request kXR_close (handle: 0x00000000) %MSG-w XrdFileWarning:  Source delayed close failed with error '[ERROR] Socket error' (errno=0, code=102, server=lxf

### 4. Rule: if there is no Fatal Exception and no Performance Kill choose the first message with exit code equal to the console exit code

In case there is neither a performance kill nor a fatal exception, choose simply the first message that has the same exit code as the error code reported by the console.

In [32]:
seq3 = 'ReportManipulatingError,CMSSWStepFailure,BadFWJRXML,ErrorLoggingAddition,WMAgentStepExecutionError'
seq4 = 'ReportManipulatingError,BadFWJRXML,CMSSWStepFailure,ErrorLoggingAddition,WMAgentStepExecutionError'
messages.test_msg(tokens, seq3, 33)


Error code: 139 Site: T2_CH_CERN


Exit code: 99996 Step: 0 Name: stageOut1 Type: ReportManipulatingError

Failed to find a step report for stageOut1!


Exit code: 139 Step: 2 Name: cmsRun1 Type: CMSSWStepFailure

  Adding last 25 lines of CMSSW stdout: #17 0x00002ac63e5963a2 in edm::WorkerT<edm::stream::EDProducerAdaptorBase>::implDo(edm::EventPrincipal const&, edm::EventSetup const&, edm::ModuleCallingContext const*) () from /cvmfs/cms.cern.ch/slc6_amd64_gcc630/cms/cmssw/CMSSW_9_4_7/lib/slc6_amd64_gcc630/libFWCoreFramework.so #18 0x00002ac63e54e217 in decltype ({parm#1}()) edm::convertException::wrap<bool edm::Worker::runModule<edm::OccurrenceTraits<edm::EventPrincipal, (edm::BranchActionType)1> >(edm::OccurrenceTraits<edm::EventPrincipal, (edm::BranchActionType)1>::MyPrincipal const&, edm::EventSetup const&, edm::StreamID, edm::ParentContext const&, edm::OccurrenceTraits<edm::EventPrincipal, (edm::BranchActionType)1>::Context const*)::{lambda()#1}>(bool edm::Worker::runModule<edm::

## Fractions of the rules

In [36]:
fatal = 'Fatal Exception'
perf = 'PerformanceKill'

In [37]:
fatal_msg = []
perf_msg = []
fatal_perf_msg = []
different = []

for i in range(len(sequences)):
    entry = sequences.iloc[i]
    seq = entry['sequences']
    counts = entry['counts']
    if perf in seq and fatal in seq:
        fatal_perf_msg.append(counts)
    elif perf in seq:
        perf_msg.append(counts)
    elif fatal in seq:
        fatal_msg.append(counts)
    else:
        different.append(counts)

In [45]:
print 'Fraction of', fatal, np.sum(fatal_msg) / float(sequences['counts'].sum())
print 'Fraction of', perf, np.sum(perf_msg) / float(sequences['counts'].sum())
print 'Fraction of', fatal, 'and', perf, np.sum(fatal_perf_msg) / float(sequences['counts'].sum())
print 'Fraction of different types', np.sum(different) / float(sequences['counts'].sum())

Fraction of Fatal Exception 0.521466449112788
Fraction of PerformanceKill 0.20487456659188252
Fraction of Fatal Exception and PerformanceKill 0.009747433544088652
Fraction of different types 0.26391155075124073
